## Adding libs

In [1]:
import os
import tqdm
import re
import tqdm
import sys
import pandas as pd
import numpy as np
from keras.utils import np_utils
from keras.models import load_model
import tensorflow as tf
from IPython.display import clear_output
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback
from keras.preprocessing.text import Tokenizer
import re
from data_loader import BooksDataLoader
from model.rnn import MultiLSTMModel
import gc
from tensorflow.keras import backend as k

# Config GPU

In [2]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession
# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
# # physical_devices = tf.config.experimental.list_physical_devices('GPU')
# # config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
# tf.compat.v1.Session(config=config)
#
# os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
# print(os.getenv('TF_GPU_ALLOCATOR'))
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.85
session = tf.compat.v1.Session(config=config)

2022-05-14 14:22:24.189512: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-14 14:22:24.241978: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-14 14:22:24.247944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-14 14:22:24.248187: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

# Loading data from directory

In [3]:
dataset = BooksDataLoader(limit=2)
dataset_loader = dataset.loader(chapter_split=1)

100%|██████████| 2/2 [00:00<00:00, 92.16it/s]


# Preprocessing

In [4]:
seq_length = 20
token_type = 'word'

In [5]:
text_dataframe = BooksDataLoader.dataframe_loader_to_text(data_loader=dataset_loader)
text_dataframe[2:3]

[['\n\n\n\n\nA NEW ACQUAINTANCE—THE STROLLER’S TALE—A DISAGREEABLE INTERRUPTION, AND AN UNPLEASANT ENCOUNTERMr. Pickwick had felt some apprehensions in consequence of the unusual absence of his two friends, which their mysterious behaviour during the whole morning had by no means tended to diminish. It was, therefore, with more than ordinary pleasure that he rose to greet them when they again entered; and with more than ordinary interest that he inquired what had occurred to detain them from his society. In reply to his questions on this point, Mr. Snodgrass was about to offer an historical account of the circumstances just now detailed, when he was suddenly checked by observing that there were present, not only Mr. Tupman and their stage-coach companion of the preceding day, but another stranger of equally singular appearance. It was a careworn-looking man, whose sallow face, and deeply-sunken eyes, were rendered still more striking than Nature had made them, by the straight black hai

In [6]:
start_story = '| ' * seq_length
def clean_text():
    clean_dataset = []
    for dataframe in text_dataframe:
        clean = ''
        for frame in dataframe:
            clean += frame
            pass

        clean = start_story + clean.strip()
        clean = clean.lower()
        # text = text.replace('\n', ' ')
        # clean = clean.replace('\n\n\n\n\n', start_story)
        clean = clean.replace('. . .', '.')
        clean = clean.replace('    ', ' ')
        clean = re.sub('\[.*?]', '', clean).strip()
        clean = clean.replace('..', '.')
        clean = clean.replace('. ', '.')
        clean = re.sub('  +', '. ', clean).strip()
        clean = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', clean)
        clean = re.sub('\s{2,}', ' ', clean)
        clean_dataset.append(clean)
        pass
    return clean_dataset

In [7]:
text_dataframe = clean_text()
text_dataframe[2:3]

[' | | | | | | | | | | | | | | | | | | | | a new acquaintance—the stroller’s tale—a disagreeable interruption , and an unpleasant encountermr . pickwick had felt some apprehensions in consequence of the unusual absence of his two friends , which their mysterious behaviour during the whole morning had by no means tended to diminish . it was , therefore , with more than ordinary pleasure that he rose to greet them when they again entered ; and with more than ordinary interest that he inquired what had occurred to detain them from his society . in reply to his questions on this point , mr . snodgrass was about to offer an historical account of the circumstances just now detailed , when he was suddenly checked by observing that there were present , not only mr . tupman and their stage - coach companion of the preceding day , but another stranger of equally singular appearance . it was a careworn - looking man , whose sallow face , and deeply - sunken eyes , were rendered still more strikin

In [8]:
if token_type == 'word':
    tokenizer = Tokenizer(char_level = False, filters = '')
else:
    tokenizer = Tokenizer(char_level = True, filters = '', lower = False)

tokenizer.fit_on_texts(text_dataframe)

total_words = len(tokenizer.word_index) + 1

token_list = tokenizer.texts_to_sequences(text_dataframe)


In [9]:
def generate_sequences(token_list, step):
    X = []
    y = []

    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])

    y = np_utils.to_categorical(y, num_classes = total_words)

    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")

    return X, y, num_seq

# Learn
### Create model

Test fit with test dataframes

In [10]:
def on_epoch_end(epoch, logs):
    gc.collect()
    k.clear_session()
    pass


model = MultiLSTMModel('Multi_LSTM_RNN_Model_2_books.h5', total_words, embedding_size=500)

step = 1
seq_length = 20
count_shapes = []
for i in range(len(text_dataframe)):
    print("-----------Started learning-----------")
    print(fr"---{i}/{len(text_dataframe)}-----------")
    X, y, num_seq = generate_sequences(token_list[i], step)


    X = np.array(X)
    y = np.array(y)
    batch_size = 32
    num_batches = int(len(X) / batch_size)
    callback = LambdaCallback(on_epoch_end=on_epoch_end)
    model.fit(X, y, epochs=50, batch_size=num_batches, callbacks=callback, save=True)
    count_shapes.append([X.shape, y.shape])
    pass


2022-05-14 14:22:25.844114: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-14 14:22:25.844432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-14 14:22:25.844668: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-14 14:22:25.845178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-14 14:22:25.845423: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 500)         9874000   
                                                                 
 lstm (LSTM)                 (None, None, 1024)        6246400   
                                                                 
 dropout (Dropout)           (None, None, 1024)        0         
                                                                 
 lstm_1 (LSTM)               (None, 1024)              8392704   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 19748)             202417

2022-05-14 14:22:30.763168: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400
Could not load symbol cublasGetSmCountTarget from libcublas.so.11. Error: /usr/local/cuda/targets/x86_64-linux/lib/libcublas.so.11: undefined symbol: cublasGetSmCountTarget


33/33 [==============================] - 7s 125ms/step - loss: 7.2958 - mae: 9.9256e-05
Epoch 2/50
33/33 [==============================] - 5s 128ms/step - loss: 5.7328 - mae: 9.9146e-05
Epoch 3/50
33/33 [==============================] - 5s 133ms/step - loss: 5.4041 - mae: 9.9045e-05
Epoch 4/50
33/33 [==============================] - 5s 127ms/step - loss: 5.1754 - mae: 9.8637e-05
Epoch 5/50
33/33 [==============================] - 5s 127ms/step - loss: 4.9616 - mae: 9.7607e-05
Epoch 6/50
33/33 [==============================] - 4s 124ms/step - loss: 4.7564 - mae: 9.6165e-05
Epoch 7/50
33/33 [==============================] - 5s 130ms/step - loss: 4.5674 - mae: 9.5198e-05
Epoch 8/50
33/33 [==============================] - 5s 130ms/step - loss: 4.3880 - mae: 9.4240e-05
Epoch 9/50
33/33 [==============================] - 5s 126ms/step - loss: 4.2941 - mae: 9.3847e-05
Epoch 10/50
33/33 [==============================] - 5s 125ms/step - loss: 4.1185 - mae: 9.2866e-05
Epoch 11/50
33/33 [=

2022-05-14 14:26:28.493163: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 934238384 exceeds 10% of free system memory.
2022-05-14 14:26:28.984605: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 934238384 exceeds 10% of free system memory.


...

Epoch 1/50
33/33 [==============================] - 14s 410ms/step - loss: 7.9170 - mae: 9.4804e-05
Epoch 2/50
33/33 [==============================] - 14s 411ms/step - loss: 5.8234 - mae: 9.2101e-05
Epoch 3/50
33/33 [==============================] - 14s 411ms/step - loss: 5.0102 - mae: 8.8161e-05
Epoch 4/50
33/33 [==============================] - 14s 411ms/step - loss: 4.2802 - mae: 8.3135e-05
Epoch 5/50
33/33 [==============================] - 14s 411ms/step - loss: 3.6050 - mae: 7.7569e-05
Epoch 6/50
33/33 [==============================] - 14s 404ms/step - loss: 2.9852 - mae: 7.1211e-05
Epoch 7/50
33/33 [==============================] - 14s 405ms/step - loss: 2.3902 - mae: 6.4428e-05
Epoch 8/50
33/33 [==============================] - 14s 404ms/step - loss: 1.9058 - mae: 5.7756e-05
Epoch 9/50
33/33 [==============================] - 14s 405ms/step - loss: 1.4707 - mae: 5.0030e-05
Epoch 10/50
33/33 [==============================] - 14s 404ms/step - loss: 1.0990 - mae: 4.2504e-05

2022-05-14 18:29:02.674085: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 913384496 exceeds 10% of free system memory.


Epoch 1/50


2022-05-14 18:29:04.007112: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 343.63M (360327936 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-05-14 18:29:04.007445: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 343.63M (360327936 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-05-14 18:29:14.009011: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 343.63M (360327936 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-05-14 18:29:14.010132: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 343.63M (360327936 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-05-14 18:29:14.010206: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 141.02MiB (rounded to 147865856)requested by op CudnnRNN
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOC

InternalError: Graph execution error:

Failed to call ThenRnnForward with model config: [rnn_mode, rnn_input_mode, rnn_direction_mode]: 2, 0, 0 , [num_layers, input_size, num_units, dir_count, max_seq_length, batch_size, cell_num_units]: [1, 1024, 1024, 1, 20, 361, 1024] 
	 [[{{node CudnnRNN}}]]
	 [[model/lstm_1/PartitionedCall]] [Op:__inference_train_function_5723]

In [ ]:
model.save()

In [2]:
def sample_with_temp(preds, temperature=1.0):

    preds = np.asarray(preds).astype('float6`4')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text

    seed_text = start_story + seed_text

    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, max_sequence_len))

        probs = model.model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)

        if y_class == 0:
            output_word = ''
        else:
            output_word = tokenizer.index_word[y_class]

        if output_word == "|":
            break

        if token_type == 'word':
            output_text += output_word + ' '
            seed_text += output_word + ' '
        else:
            output_text += output_word + ' '
            seed_text += output_word + ' '


    return output_text

In [12]:

train_model = False
if train_model:
    epochs = 1
    batch_size = 32
    num_batches = int(len(X) / batch_size)
    callback = LambdaCallback(on_epoch_end=on_epoch_end)
    model.fit(X, y, epochs=epochs, batch_size=batch_size, shuffle = True)
    # model.partial_fit(X, y, epochs=epochs, batch_size=batch_size, shuffle = True)
    model.save(os.path.dirname(os.path.abspath(os.getcwd())) + "/models/rnn_model.h5")
    print("Model saved")

In [5]:
seed_text = "When you can to say  "
gen_words = 100
temp = 1.0

print (generate_text(seed_text, gen_words, model.model, seq_length, temp))

NameError: name 'model' is not defined